# 03 - ROC Markers & Comparison

In [ ]:

import scanpy as sc, pandas as pd, numpy as np, matplotlib.pyplot as plt, os
adata = sc.read_h5ad("results/preprocessed.h5ad")
if 'tissue' in adata.obs.columns and 'Skin' in set(adata.obs['tissue']):
    skin = adata[adata.obs['tissue'] == 'Skin'].copy()
else:
    skin = adata.copy()
sc.pp.neighbors(skin, n_neighbors=15, n_pcs=min(30, skin.obsm['X_pca'].shape[1]))
sc.tl.umap(skin)
sc.tl.leiden(skin, resolution=0.8, key_added='leiden_skin')
sc.pl.umap(skin, color=['leiden_skin'], legend_loc='on data', show=False)
plt.savefig("results/figures/umap_skin_clusters.png", bbox_inches="tight")
sc.tl.rank_genes_groups(skin, 'leiden_skin', method='logreg', key_added='rank_logreg')
sc.tl.rank_genes_groups(skin, 'leiden_skin', method='wilcoxon', key_added='rank_wilcoxon')
putative = ['fgf8','wnt5b','wnt3a','sox2','sox10','ctgfa','mki67']
genes_exist = [g for g in putative if g in skin.var_names]
if genes_exist:
    sc.pp.scale(skin, max_value=10, copy=False)
    skin.obs['roc_score'] = skin[:, genes_exist].X.mean(axis=1)
    roc_cluster = skin.obs.groupby('leiden_skin')['roc_score'].mean().idxmax()
else:
    roc_cluster = skin.obs['leiden_skin'].value_counts().idxmax()
def top_markers(adata_obj, key, group, n=100):
    names = pd.DataFrame(adata_obj.uns[key]['names'])[group][:n].tolist()
    scores = pd.DataFrame(adata_obj.uns[key]['scores'])[group][:n].tolist()
    return pd.DataFrame({'gene': names, 'score': scores})
logreg_df = top_markers(skin, 'rank_logreg', roc_cluster, n=100)
wilc_df = top_markers(skin, 'rank_wilcoxon', roc_cluster, n=100)
merged = logreg_df.merge(wilc_df, on='gene', how='outer', suffixes=('_logreg','_wilcoxon'))
merged.to_csv("results/roc_markers_logreg_wilcoxon.csv", index=False)
consensus = merged.dropna(subset=['score_logreg','score_wilcoxon']).sort_values('score_logreg', ascending=False).head(12)['gene'].tolist()
if consensus:
    sc.pl.dotplot(skin, consensus, groupby='leiden_skin', show=False)
    plt.savefig("results/figures/roc_consensus.png", bbox_inches="tight")
    sc.pl.violin(skin, consensus[:6], groupby='leiden_skin', stripplot=False, multi_panel=True, show=False)
    plt.savefig("results/figures/roc_violin.png", bbox_inches="tight")
sup = "data/Supplementary_Table3.csv"
if os.path.exists(sup):
    sup3 = pd.read_csv(sup)
else:
    sup3 = pd.DataFrame(columns=['gene'])
roc_set = set(merged['gene'].dropna().tolist())
sup_set = set(sup3['gene'].astype(str).str.strip())
overlap = sorted(roc_set.intersection(sup_set))
pd.Series(overlap, name='overlap_gene').to_csv("results/overlap_with_supp_table3.csv", index=False)
skin.write_h5ad("results/skin_subset.h5ad")
print("ok")
